Nama: : Jessica Theresia<br>
Email: : me.jessicatheresia@gmail.com<br>
ID Dicoding: : jessica_trs<br>

# **1. Import Library**

Tahap untuk mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV di google drive ke dalam variabel DataFrame.

In [2]:
# Mount Google Drive
drive.mount('/content/drive')

# Memuat dataset
df = pd.read_csv("/content/drive/MyDrive/Dicoding ML Submission/Belajar Machine Learning untuk Pemula/Dataset_inisiasi.csv")

# Menampilkan 5 baris pertama
print(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   TransactionAmount  TransactionType  Location  DeviceID  MerchantID  \
0          -1.693251                1        36       365          14   
1           0.576066                1        15        50          51   
2          -0.378893                1        23       229           8   
3          -0.107205                1        33       182           1   
4          -1.722670                0         1       298          90   

   Channel  CustomerAge  CustomerOccupation  TransactionDuration  \
0        0         70.0                   0                 81.0   
1        0         68.0                   0                141.0   
2        2         19.0                   3                 56.0   
3        2         26.0                   3                 25.0   
4        2         26.0                   3                198.0   

   LoginAttempts  Accou

## Informasi Dataset

untuk memastikan bahwa dataset sudah dilakukan clustering.

In [3]:
# Menampilkan informasi dasar dataset
dataset_info = df.info()
dataset_head = df.head()
missing_values = df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2512 entries, 0 to 2511
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TransactionAmount    2512 non-null   float64
 1   TransactionType      2512 non-null   int64  
 2   Location             2512 non-null   int64  
 3   DeviceID             2512 non-null   int64  
 4   MerchantID           2512 non-null   int64  
 5   Channel              2512 non-null   int64  
 6   CustomerAge          2512 non-null   float64
 7   CustomerOccupation   2512 non-null   int64  
 8   TransactionDuration  2512 non-null   float64
 9   LoginAttempts        2512 non-null   float64
 10  AccountBalance       2512 non-null   float64
 11  Cluster_KMeans       2512 non-null   int64  
dtypes: float64(5), int64(7)
memory usage: 235.6 KB


In [4]:
dataset_head

,TransactionAmount,TransactionType,Location,DeviceID,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,Cluster_KMeans
0,-1.693251,1,36,365,14,0,70.0,0,81.0,0.038582,5112.21,1
1,0.576066,1,15,50,51,0,68.0,0,141.0,0.038582,13758.91,0
2,-0.378893,1,23,229,8,2,19.0,3,56.0,0.038582,1122.35,0
3,-0.107205,1,33,182,1,2,26.0,3,25.0,0.038582,8569.06,0
4,-1.722670,0,1,298,90,2,26.0,3,198.0,0.038582,7429.40,0


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [5]:
# 1. Memisahkan fitur (X) dan target (y)
X = df.drop("Cluster_KMeans", axis=1)  # Semua kolom kecuali target
y = df["Cluster_KMeans"]               # Target label

# 2. Encoding fitur kategorikal (One-Hot Encoding)
X = pd.get_dummies(X, drop_first=True)

# 3. Membagi dataset menjadi training (80%) dan testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Menampilkan hasil pembagian data
X_train.shape, X_test.shape, y_train.value_counts(), y_test.value_counts()

((2009, 11),
 (503, 11),
 Cluster_KMeans
 0    1010
 1     999
 Name: count, dtype: int64,
 Cluster_KMeans
 0    253
 1    250
 Name: count, dtype: int64)

### Penjelasan Output

* (2009, 833): Training Set memiliki 2009 sampel (baris) dan 833 fitur (kolom) setelah dilakukan one-hot encoding pada fitur kategorikal.

* (503, 833): Testing Set memiliki 503 sampel (baris) dan 833 fitur (kolom) dengan struktur yang sama seperti training set.

* Distribusi Label di Training Set: 1010 sampel termasuk dalam kelas 1 dan 999 sampel termasuk dalam kelas 0. Seimbang (hampir 50:50), yang bagus untuk mencegah bias terhadap salah satu kelas.

* Distribusi Label di Testing Set: 253 sampel termasuk dalam kelas 1 dan 250 sampel termasuk dalam kelas 0. Distribusi di testing set juga seimbang, memastikan evaluasi yang adil untuk kedua kelas.

* **Kesimpulan**: Data telah berhasil dibagi menjadi training (80%) dan testing (20%). Distribusi label antara kelas 0 dan kelas 1 seimbang, yang baik untuk performa model klasifikasi.

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

In [6]:
# 1. Membangun Model Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

Penjelasan

1. Random Forest
<br>
Random Forest adalah algoritma ensemble yang terdiri dari banyak Decision Tree yang dilatih secara independen menggunakan subset data dan fitur yang berbeda. Hasil prediksi diambil berdasarkan mayoritas suara (voting) untuk klasifikasi.

* Alasan Memilih Algoritma ini :

  * a. Robust terhadap Outlier dan Noise: Karena menggunakan hasil agregasi dari banyak pohon keputusan, model ini lebih tahan terhadap data yang memiliki outlier (yang sudah kita tangani sebelumnya).

  * b. Menangkap Interaksi Fitur: Dataset kita memiliki beberapa fitur numerik yang kompleks. Random Forest dapat memahami hubungan non-linear antara fitur.

  * c. Stabilitas dan Akurasi: Dengan membangun banyak pohon, model menjadi lebih stabil dan memiliki bias rendah serta varian yang moderat.

* Kaitan dengan Dataset:
  * Dataset hasil clustering mengandung variabel numerik seperti TransactionAmount dan LoginAttempts yang memiliki distribusi yang bervariasi. Random Forest efektif dalam menangkap pola kompleks dari fitur-fitur ini.* Cocok untuk dataset dengan jumlah fitur yang tidak terlalu besar dan sudah bersih setelah penanganan outlier dan fitur redundant.

In [7]:
# 2. Membangun Model Logistic Regression
lr_model = LogisticRegression(max_iter=100, random_state=42)
lr_model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

Penjelasan

2. Logistic Regression
<br>
Logistic Regression adalah metode klasifikasi yang menggunakan fungsi sigmoid untuk memodelkan probabilitas hasil biner atau multiklas.

* Alasan Memilih Logistic Regression?

  * Sederhana dan Efisien: Algoritma ini mudah diinterpretasikan dan efisien dalam komputasi meskipun dataset memiliki banyak sampel.

  * Hubungan Linear: Jika hubungan antara fitur dan target bersifat linier, Logistic Regression akan memberikan hasil yang baik.

  * Cocok untuk Dataset Terstruktur: Logistic Regression sangat baik dalam memproses dataset yang telah dibersihkan dari outlier dan redundan.

* Kaitan dengan Dataset:

  * Dataset ini memiliki label biner hasil dari proses clustering (Cluster_KMeans: 0 atau 1), yang cocok dengan fungsi sigmoid Logistic Regression.
  * Dengan fitur yang telah ditangani outlier-nya, Logistic Regression bisa memberikan hasil akurat tanpa bias dari data ekstrem.

## **b. Evaluasi Model Klasifikasi**

**Tahapan Evaluasi Model Klasifikasi yang Dibuat untuk Melakukan Prediksi**

1. Menggunakan model Random Forest dan Logistic Regression untuk memprediksi label pada data uji (X_test),

2. Menghitung Metrik Evaluasi
  * Accuracy: Persentase prediksi yang benar dari total data uji.
  * F1-Score: Rata-rata harmonik dari Precision
  * Recall untuk mengukur keseimbangan prediksi.

3. Membuat Confusion Matrix

  * Menampilkan jumlah True Positive (TP), True Negative (TN), False Positive (FP), dan False Negative (FN) untuk kedua model.

4. Laporan Klasifikasi (Classification Report)

  * Menyediakan metrik lengkap: Precision, Recall, dan F1-Score untuk masing-masing kelas.

5. Membandingkan Performa Model

  * Menyusun hasil evaluasi dalam bentuk tabel untuk membandingkan performa Random Forest dan Logistic Regression.

In [8]:
# 1. Melakukan prediksi menggunakan data uji
rf_preds = rf_model.predict(X_test)  # Prediksi Random Forest
lr_preds = lr_model.predict(X_test)  # Prediksi Logistic Regression

# 2. Metrik evaluasi (Accuracy dan F1-Score)
rf_accuracy = accuracy_score(y_test, rf_preds)
rf_f1 = f1_score(y_test, rf_preds)

lr_accuracy = accuracy_score(y_test, lr_preds)
lr_f1 = f1_score(y_test, lr_preds)

# 3. Confusion matrix
rf_conf_matrix = confusion_matrix(y_test, rf_preds)
lr_conf_matrix = confusion_matrix(y_test, lr_preds)

# 4. Precision, Recall, F1-Score
rf_report = classification_report(y_test, rf_preds)
lr_report = classification_report(y_test, lr_preds)

In [9]:
# Membuat DataFrame untuk hasil evaluasi
results = pd.DataFrame({
    "Model": ["Random Forest", "Logistic Regression"],
    "Accuracy": [rf_accuracy, lr_accuracy],
    "F1-Score": [rf_f1, lr_f1]
})

In [10]:
# Menampilkan hasil evaluasi dalam tabel
print("=== Hasil Evaluasi Model ===")
print(results.to_string(index=False))

=== Hasil Evaluasi Model ===
              Model  Accuracy  F1-Score
      Random Forest  1.000000  1.000000
Logistic Regression  0.918489  0.921002


In [11]:
# Perbandingan confusion matrix
print("\n=== Confusion Matrix - Random Forest ===")
print(rf_conf_matrix)

print("\n=== Confusion Matrix - Logistic Regression ===")
print(lr_conf_matrix)


=== Confusion Matrix - Random Forest ===
[[253   0]
 [  0 250]]

=== Confusion Matrix - Logistic Regression ===
[[223  30]
 [ 11 239]]


In [12]:
# Perbandingan Classification Report
print("\n=== Classification Report - Random Forest ===")
print(rf_report)

print("\n=== Classification Report - Logistic Regression ===")
print(lr_report)


=== Classification Report - Random Forest ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       253
           1       1.00      1.00      1.00       250

    accuracy                           1.00       503
   macro avg       1.00      1.00      1.00       503
weighted avg       1.00      1.00      1.00       503


=== Classification Report - Logistic Regression ===
              precision    recall  f1-score   support

           0       0.95      0.88      0.92       253
           1       0.89      0.96      0.92       250

    accuracy                           0.92       503
   macro avg       0.92      0.92      0.92       503
weighted avg       0.92      0.92      0.92       503



Penjelasan :<br>

1. Confusion Matrix
  * **Random Forest**
    * Random Forest memiliki performa sempurna (akurasi 100%) dengan tidak ada kesalahan klasifikasi (semua prediksi benar). Ini menunjukkan bahwa model sangat kuat dalam menangani data yang digunakan.

  * **Logistic Regression**
    * 30 false negative (FN) → Model memprediksi negatif padahal seharusnya positif.
    * 11 false positive (FP) → Model memprediksi positif padahal seharusnya negatif. <br>
  Ini menunjukkan model masih memiliki ruang untuk perbaikan sebelum tuning.


2. Classification Report
     * Kinerja Per Kelas
    * **Random Forest**
        * Random Forest mencapai performa sempurna (1.00) pada precision, recall, dan F1-score, menunjukkan model ini tidak membuat kesalahan klasifikasi sama sekali. Ini mengindikasikan model sangat cocok dengan data, tetapi perlu dievaluasi lebih lanjut untuk memastikan tidak terjadi overfitting.
    * **Logistic Regression**
        * memiliki precision lebih tinggi untuk kelas 0 (95%), tetapi recall lebih rendah (88%), yang berarti masih ada beberapa sampel positif yang diprediksi sebagai negatif.
        * Sebaliknya, untuk kelas 1, recall tinggi (96%) tetapi precision lebih rendah (89%), artinya model lebih cenderung mengklasifikasikan sampel sebagai positif, meskipun ada beberapa false positive.
        * Secara keseluruhan, Logistic Regression sebelum tuning masih cukup baik (F1-score = 0.92), tetapi setelah tuning mengalami peningkatan signifikan (99,4% akurasi), yang berarti model lebih optimal dalam menangkap pola data.


3. Analisis Model

  * Random Forest memiliki performa sempurna dengan precision, recall, dan F1-score mencapai 1.00. Namun, perlu diperhatikan kemungkinan overfitting, terutama jika model terlalu menyesuaikan dengan data latih dan kurang mampu menggeneralisasi pada data baru.
  * Logistic Regression (sebelum tuning) masih menunjukkan kelemahan dalam recall kelas 0 (hanya 0.88), yang berarti model ini lebih sering salah dalam mendeteksi kelas negatif. Namun, setelah tuning, performanya meningkat signifikan dengan akurasi 99,4%, yang menunjukkan keseimbangan precision dan recall yang lebih baik untuk kedua kelas.

<br>
**Kesimpulan** <br>
Logistic Regression menunjukkan performa yang jauh lebih baik dibandingkan Random Forest dalam hal:

* Logistic Regression menunjukkan performa yang lebih baik dibandingkan Random Forest dalam beberapa aspek berikut:
* Akurasi lebih tinggi setelah tuning (99,4% vs 100%), dengan peningkatan signifikan dibandingkan sebelum tuning (91,8% → 99,4%).
* Kesalahan prediksi lebih rendah, ditunjukkan dengan penurunan jumlah False Positive dan False Negative setelah tuning.
* Hubungan linier antara fitur dan label dalam dataset semakin terlihat setelah menangani outlier dan fitur redundan, yang membuat Logistic Regression lebih optimal.
* Lebih cocok untuk klasifikasi biner dengan pola linier, seperti pada dataset ini, sehingga dapat memberikan prediksi yang lebih stabil dan mudah diinterpretasikan dibandingkan Random Forest yang berisiko overfitting.

## **c. Tuning Model Klasifikasi (Optional)**

Tuning dilakukan untuk mencari kombinasi parameter terbaik yang dapat meningkatkan performa model klasifikasi yang masih bisa ditingkatkan yaitu Logistic Regession. Metode yang digunakan adalah GridSearchCV, yang mencoba berbagai kombinasi parameter dengan validasi silang (cross-validation) untuk menghindari overfitting dan mendapatkan model optimal.

In [13]:
# Hyperparameter tuning Random Forest
rf_params = {
    "n_estimators": [30, 40, 50],  # Kurangi jumlah estimator
    "max_depth": [5, 10, None],  # Batasi kedalaman pohon
    "min_samples_split": [5, 10],  # Minimum sampel untuk split cabang
    "min_samples_leaf": [2, 5]  # Minimum sampel dalam daun
}

# GridSearchCV untuk mencari kombinasi terbaik
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=10, scoring="f1", n_jobs=-1)
rf_grid.fit(X_train, y_train)

# Model terbaik setelah tuning
best_rf_model = rf_grid.best_estimator_

# Evaluasi ulang setelah tuning
rf_tuned_preds = best_rf_model.predict(X_test)
rf_tuned_accuracy = accuracy_score(y_test, rf_tuned_preds)
rf_tuned_f1 = f1_score(y_test, rf_tuned_preds)

# Menampilkan hasil tuning
print("=== Hasil Tuning Random Forest ===")
print(f"Best Params: {rf_grid.best_params_}")
print(f"Testing Accuracy : {rf_tuned_accuracy:.4f}")
print(f"Testing F1-Score : {rf_tuned_f1:.4f}")

=== Hasil Tuning Random Forest ===
Best Params: {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 30}
Testing Accuracy : 1.0000
Testing F1-Score : 1.0000


In [14]:
# Hyperparameter tuning untuk Logistic Regression
lr_params = {
    "C": [0.01, 0.1, 1, 10, 100],  # Regularisasi
    "solver": ["liblinear", "lbfgs"]  # Solver optimasi
}

# GridSearchCV untuk mencari kombinasi terbaik
lr_grid = GridSearchCV(LogisticRegression(max_iter=200, random_state=42), lr_params, cv=5, scoring="f1", n_jobs=-1)
lr_grid.fit(X_train, y_train)

# Model terbaik setelah tuning
best_lr_model = lr_grid.best_estimator_

# Evaluasi ulang Logistic Regression setelah tuning
lr_tuned_preds = best_lr_model.predict(X_test)
lr_tuned_accuracy = accuracy_score(y_test, lr_tuned_preds)
lr_tuned_f1 = f1_score(y_test, lr_tuned_preds)

# Menampilkan hasil tuning
print("=== Hasil Tuning Logistic Regression ===")
print(f"Best Params: {lr_grid.best_params_}")
print(f"Testing Accuracy : {lr_tuned_accuracy:.4f}")
print(f"Testing F1-Score : {lr_tuned_f1:.4f}")

=== Hasil Tuning Logistic Regression ===
Best Params: {'C': 100, 'solver': 'liblinear'}
Testing Accuracy : 0.9940
Testing F1-Score : 0.9940


In [15]:
# Menyusun hasil evaluasi dalam DataFrame
results_tuning = pd.DataFrame({
    "Model": ["Random Forest (Before)", "Random Forest (After)",
              "Logistic Regression (Before)", "Logistic Regression (After)"],
    "Accuracy": [rf_accuracy, rf_tuned_accuracy, lr_accuracy, lr_tuned_accuracy],
    "F1-Score": [rf_f1, rf_tuned_f1, lr_f1, lr_tuned_f1]
})

# Menampilkan hasil evaluasi setelah tuning
print("=== Hasil Evaluasi Model Sebelum & Sesudah Tuning ===")
print(results_tuning.to_string(index=False))

=== Hasil Evaluasi Model Sebelum & Sesudah Tuning ===
                       Model  Accuracy  F1-Score
      Random Forest (Before)  1.000000  1.000000
       Random Forest (After)  1.000000  1.000000
Logistic Regression (Before)  0.918489  0.921002
 Logistic Regression (After)  0.994036  0.994012


Penjelasan
* Random Forest sudah optimal sejak awal, sehingga tuning tidak memberikan perubahan signifikan.
* Logistic Regression mengalami peningkatan setelah tuning, menunjukkan bahwa pemilihan parameter yang lebih optimal sangat berpengaruh pada model ini.
* Tuning lebih berdampak pada model berbasis regresi dibandingkan dengan model berbasis pohon keputusan.

## **d. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

## Logistic Regression

Perbandingan hasil evaluasi sebelum dan sesudah tuning untuk logistic regression

In [16]:
# 1. Evaluasi sebelum tuning
lr_accuracy_before = lr_accuracy
lr_f1_before = lr_f1
lr_report_before = classification_report(y_test, lr_preds, output_dict=True)

# 2. Evaluasi setelah tuning
lr_accuracy_after = accuracy_score(y_test, lr_tuned_preds)
lr_f1_after = f1_score(y_test, lr_tuned_preds)
lr_report_after = classification_report(y_test, lr_tuned_preds, output_dict=True)

# 3. Menghitung peningkatan performa
acc_improvement = (lr_accuracy_after - lr_accuracy_before) * 100
f1_improvement = (lr_f1_after - lr_f1_before) * 100

In [17]:
# 4. Menampilkan hasil perbandingan sebelum & sesudah tuning
print("=== Perbandingan Performa Logistic Regression ===")
print(f"Accuracy Sebelum Tuning : {lr_accuracy_before:.4f} ({lr_accuracy_before * 100:.2f}%)")
print(f"Accuracy Setelah Tuning : {lr_accuracy_after:.4f} ({lr_accuracy_after * 100:.2f}%)")
print(f"Peningkatan Accuracy    : {acc_improvement:.2f}%\n")

print(f"F1-Score Sebelum Tuning : {lr_f1_before:.4f} ({lr_f1_before * 100:.2f}%)")
print(f"F1-Score Setelah Tuning : {lr_f1_after:.4f} ({lr_f1_after * 100:.2f}%)")
print(f"Peningkatan F1-Score    : {f1_improvement:.2f}%\n")

# 5. Identifikasi kelemahan model berdasarkan Classification Report
print("\n=== Analisis Kelemahan Model ===")
for label in ['0', '1']:
    precision_before = lr_report_before[label]['precision']
    recall_before = lr_report_before[label]['recall']
    precision_after = lr_report_after[label]['precision']
    recall_after = lr_report_after[label]['recall']

    print(f"Kelas {label}:")
    print(f"  - Precision Sebelum: {precision_before:.4f}, Setelah: {precision_after:.4f}")
    print(f"  - Recall Sebelum   : {recall_before:.4f}, Setelah: {recall_after:.4f}")

    if precision_after < 0.8 or recall_after < 0.8:
        print(f"Precision/Recall masih di bawah 0.8 untuk kelas {label}. Bisa jadi ada ketidakseimbangan data.")

# 6. Rekomendasi tindakan lanjutan
print("\n=== Rekomendasi Tindakan Lanjutan ===")
if acc_improvement < 1.0 and f1_improvement < 1.0:
    print("Peningkatan performa kecil. Pertimbangkan metode lain seperti menambah fitur atau balancing data.")
if any(lr_report_after[label]['recall'] < 0.8 for label in ['0', '1']):
    print("Recall masih rendah untuk beberapa kelas. Coba data augmentation atau algoritma lain.")
else :
  print("Model sudah sangat baik. Bisa digunakan untuk deployment.")
# 7. Menampilkan hasil dalam bentuk DataFrame
comparison_df = pd.DataFrame({
    'Metric': ['Accuracy', 'F1-Score'],
    'Before Tuning': [lr_accuracy_before, lr_f1_before],
    'After Tuning': [lr_accuracy_after, lr_f1_after],
    'Improvement (%)': [acc_improvement, f1_improvement]
})

print("\n=== Ringkasan Hasil Evaluasi ===")
print(comparison_df)

=== Perbandingan Performa Logistic Regression ===
Accuracy Sebelum Tuning : 0.9185 (91.85%)
Accuracy Setelah Tuning : 0.9940 (99.40%)
Peningkatan Accuracy    : 7.55%

F1-Score Sebelum Tuning : 0.9210 (92.10%)
F1-Score Setelah Tuning : 0.9940 (99.40%)
Peningkatan F1-Score    : 7.30%


=== Analisis Kelemahan Model ===
Kelas 0:
  - Precision Sebelum: 0.9530, Setelah: 0.9960
  - Recall Sebelum   : 0.8814, Setelah: 0.9921
Kelas 1:
  - Precision Sebelum: 0.8885, Setelah: 0.9920
  - Recall Sebelum   : 0.9560, Setelah: 0.9960

=== Rekomendasi Tindakan Lanjutan ===
Model sudah sangat baik. Bisa digunakan untuk deployment.

=== Ringkasan Hasil Evaluasi ===
     Metric  Before Tuning  After Tuning  Improvement (%)
0  Accuracy       0.918489      0.994036         7.554672
1  F1-Score       0.921002      0.994012         7.301005


Cek Overfitting atau Underfitting

In [18]:
# Evaluasi Logistic Regression di TRAINING SET
lr_train_preds = best_lr_model.predict(X_train)
lr_train_accuracy = accuracy_score(y_train, lr_train_preds)
lr_train_f1 = f1_score(y_train, lr_train_preds)

# Evaluasi Logistic Regression di TESTING SET
lr_test_preds = best_lr_model.predict(X_test)
lr_test_accuracy = accuracy_score(y_test, lr_test_preds)
lr_test_f1 = f1_score(y_test, lr_test_preds)

# Menampilkan hasil evaluasi di kedua set
print("=== Evaluasi Overfitting & Underfitting Model Logistic Regression ===")
print(f"Training Accuracy : {lr_train_accuracy:.4f}")
print(f"Training F1-Score : {lr_train_f1:.4f}")
print(f"Testing Accuracy  : {lr_test_accuracy:.4f}")
print(f"Testing F1-Score  : {lr_test_f1:.4f}")

# Cek perbedaan performa
diff_accuracy = lr_train_accuracy - lr_test_accuracy
diff_f1 = lr_train_f1 - lr_test_f1
print(f"\nPerbedaan Accuracy : {diff_accuracy:.4f}")
print(f"Perbedaan F1-Score : {diff_f1:.4f}")

# Analisis Overfitting atau Underfitting
if diff_accuracy > 0.05 or diff_f1 > 0.05:
    print("\nModel kemungkinan mengalami OVERFITTING. Performa pada training jauh lebih tinggi dibanding testing.")
elif lr_test_accuracy < 0.80 or lr_test_f1 < 0.80:
    print("\nModel kemungkinan mengalami UNDERFITTING. Performa di testing set masih kurang baik.")
else:
    print("\nModel memiliki generalisasi yang baik. Tidak ada indikasi overfitting atau underfitting yang signifikan.")

=== Evaluasi Overfitting & Underfitting Model Logistic Regression ===
Training Accuracy : 0.9950
Training F1-Score : 0.9950
Testing Accuracy  : 0.9940
Testing F1-Score  : 0.9940

Perbedaan Accuracy : 0.0010
Perbedaan F1-Score : 0.0010

Model memiliki generalisasi yang baik. Tidak ada indikasi overfitting atau underfitting yang signifikan.


## Random Forest

In [19]:
# 1. Evaluasi sebelum tuning
rf_accuracy_before = rf_accuracy
rf_f1_before = rf_f1
rf_report_before = classification_report(y_test, rf_preds, output_dict=True)

# 2. Evaluasi setelah tuning
rf_accuracy_after = accuracy_score(y_test, rf_tuned_preds)
rf_f1_after = f1_score(y_test, rf_tuned_preds)
rf_report_after = classification_report(y_test, rf_tuned_preds, output_dict=True)

# 3. Menghitung peningkatan performa
rf_acc_improvement = (rf_accuracy_after - rf_accuracy_before) * 100
rf_f1_improvement = (rf_f1_after - rf_f1_before) * 100

In [20]:
# 4. Menampilkan hasil perbandingan sebelum & sesudah tuning
print("=== Perbandingan Performa Random Forest ===")
print(f"Accuracy Sebelum Tuning : {rf_accuracy_before:.4f} ({rf_accuracy_before * 100:.2f}%)")
print(f"Accuracy Setelah Tuning : {rf_accuracy_after:.4f} ({rf_accuracy_after * 100:.2f}%)")
print(f"Peningkatan Accuracy    : {rf_acc_improvement:.2f}%\n")

print(f"F1-Score Sebelum Tuning : {rf_f1_before:.4f} ({rf_f1_before * 100:.2f}%)")
print(f"F1-Score Setelah Tuning : {rf_f1_after:.4f} ({rf_f1_after * 100:.2f}%)")
print(f"Peningkatan F1-Score    : {rf_f1_improvement:.2f}%\n")

# 5. Identifikasi kelemahan model berdasarkan Classification Report
print("\n=== Analisis Kelemahan Model Random Forest ===")
for label in ['0', '1']:
    precision_before = rf_report_before[label]['precision']
    recall_before = rf_report_before[label]['recall']
    precision_after = rf_report_after[label]['precision']
    recall_after = rf_report_after[label]['recall']

    print(f"Kelas {label}:")
    print(f"  - Precision Sebelum: {precision_before:.4f}, Setelah: {precision_after:.4f}")
    print(f"  - Recall Sebelum   : {recall_before:.4f}, Setelah: {recall_after:.4f}")

    if precision_after < 0.8 or recall_after < 0.8:
        print(f"Precision/Recall masih di bawah 0.8 untuk kelas {label}. Bisa jadi ada ketidakseimbangan data.")

# 6. Rekomendasi tindakan lanjutan
print("\n=== Rekomendasi Tindakan Lanjutan Random Forest ===")
if rf_acc_improvement < 1.0 and rf_f1_improvement < 1.0:
    print("Peningkatan performa kecil. Pertimbangkan metode lain seperti menambah fitur atau balancing data.")
if any(rf_report_after[label]['recall'] < 0.8 for label in ['0', '1']):
    print("Recall masih rendah untuk beberapa kelas. Coba data augmentation atau algoritma lain.")
else:
    print("Model sudah sangat baik. Bisa digunakan untuk deployment.")

# 7. Menampilkan hasil dalam bentuk DataFrame
rf_comparison_df = pd.DataFrame({
    'Metric': ['Accuracy', 'F1-Score'],
    'Before Tuning': [rf_accuracy_before, rf_f1_before],
    'After Tuning': [rf_accuracy_after, rf_f1_after],
    'Improvement (%)': [rf_acc_improvement, rf_f1_improvement]
})

print("\n=== Ringkasan Hasil Evaluasi Random Forest ===")
print(rf_comparison_df)

=== Perbandingan Performa Random Forest ===
Accuracy Sebelum Tuning : 1.0000 (100.00%)
Accuracy Setelah Tuning : 1.0000 (100.00%)
Peningkatan Accuracy    : 0.00%

F1-Score Sebelum Tuning : 1.0000 (100.00%)
F1-Score Setelah Tuning : 1.0000 (100.00%)
Peningkatan F1-Score    : 0.00%


=== Analisis Kelemahan Model Random Forest ===
Kelas 0:
  - Precision Sebelum: 1.0000, Setelah: 1.0000
  - Recall Sebelum   : 1.0000, Setelah: 1.0000
Kelas 1:
  - Precision Sebelum: 1.0000, Setelah: 1.0000
  - Recall Sebelum   : 1.0000, Setelah: 1.0000

=== Rekomendasi Tindakan Lanjutan Random Forest ===
Peningkatan performa kecil. Pertimbangkan metode lain seperti menambah fitur atau balancing data.
Model sudah sangat baik. Bisa digunakan untuk deployment.

=== Ringkasan Hasil Evaluasi Random Forest ===
     Metric  Before Tuning  After Tuning  Improvement (%)
0  Accuracy            1.0           1.0              0.0
1  F1-Score            1.0           1.0              0.0


In [21]:
# Evaluasi Random Forest di TRAINING SET
rf_train_preds = rf_model.predict(X_train)
rf_train_accuracy = accuracy_score(y_train, rf_train_preds)
rf_train_f1 = f1_score(y_train, rf_train_preds)

# Evaluasi Random Forest di TESTING SET
rf_test_preds = rf_model.predict(X_test)
rf_test_accuracy = accuracy_score(y_test, rf_test_preds)
rf_test_f1 = f1_score(y_test, rf_test_preds)

# Menampilkan hasil evaluasi di kedua set
print("=== Evaluasi Overfitting & Underfitting Model Random Forest ===")
print(f"Training Accuracy : {rf_train_accuracy:.4f}")
print(f"Training F1-Score : {rf_train_f1:.4f}")
print(f"Testing Accuracy  : {rf_test_accuracy:.4f}")
print(f"Testing F1-Score  : {rf_test_f1:.4f}")

# Cek perbedaan performa
diff_accuracy = rf_train_accuracy - rf_test_accuracy
diff_f1 = rf_train_f1 - rf_test_f1
print(f"\nPerbedaan Accuracy : {diff_accuracy:.4f}")
print(f"Perbedaan F1-Score : {diff_f1:.4f}")

# Analisis Overfitting atau Underfitting
if diff_accuracy > 0.05 or diff_f1 > 0.05:
    print("\nModel kemungkinan mengalami OVERFITTING. Performa pada training jauh lebih tinggi dibanding testing.")
elif rf_test_accuracy < 0.80 or rf_test_f1 < 0.80:
    print("\nModel kemungkinan mengalami UNDERFITTING. Performa di testing set masih kurang baik.")
else:
    print("\nModel memiliki generalisasi yang baik. Tidak ada indikasi overfitting atau underfitting yang signifikan.")

=== Evaluasi Overfitting & Underfitting Model Random Forest ===
Training Accuracy : 1.0000
Training F1-Score : 1.0000
Testing Accuracy  : 1.0000
Testing F1-Score  : 1.0000

Perbedaan Accuracy : 0.0000
Perbedaan F1-Score : 0.0000

Model memiliki generalisasi yang baik. Tidak ada indikasi overfitting atau underfitting yang signifikan.


## **KESIMPULAN AKHIR**

1. Performa Model Setelah Tuning

  * Logistic Regression mengalami peningkatan signifikan, dengan akurasi naik dari 91,85% menjadi 99,40% dan F1-score meningkat dari 92,10% menjadi 99,40%.
  * Random Forest tetap memiliki akurasi sempurna (100%) baik sebelum maupun setelah tuning, menunjukkan model ini sudah sangat optimal untuk dataset ini.

2. Evaluasi Overfitting

  * Logistic Regression menunjukkan generalisasi yang sangat baik, dengan selisih yang sangat kecil antara akurasi training dan testing (0,001 atau 0,1%), sehingga tidak ada indikasi overfitting.
  * Random Forest memiliki performa sempurna (100%), tetapi hal ini perlu diwaspadai karena berpotensi overfitting jika digunakan pada dataset lain.

3. Upaya Mengatasi Overfitting

  Dalam proses tuning model Random Forest, telah dilakukan beberapa strategi untuk mengurangi kemungkinan overfitting:

  * Untuk Random Forest, dilakukan berbagai strategi seperti:
  * Mengurangi jumlah estimator untuk menghindari model yang terlalu kompleks.
  * Membatasi kedalaman pohon keputusan agar tidak menangkap noise dari data.
  * Menyesuaikan minimum sampel untuk split dan leaf agar model lebih generalisasi.
  * Validasi silang (10-fold cross-validation) memastikan model bekerja konsisten di berbagai subset data.
  * Hasilnya, tidak ditemukan indikasi overfitting yang signifikan pada kedua model setelah tuning.

4. Rekomendasi Penggunaan Model<br>
* **Gunakan Logistic Regression untuk deployment karena:**
  * Model lebih sederhana dan tidak rentan terhadap overfitting
  * Performa sudah sangat tinggi (99,40%) setelah tuning.
  * Cocok untuk kasus klasifikasi biner dengan pola linier seperti pada dataset ini.

* ** Gunakan Random Forest ** jika dataset tetap dan terkontrol, namun perlu dipantau jika diterapkan pada dataset lain untuk menghindari overfitting.
  * Jika model digunakan untuk dataset berbeda, pertimbangkan penyesuaian parameter atau metode lain seperti balancing data agar tetap optimal.


